In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iris-dataset-extended/iris_extended.csv


In [2]:
import pandas as pd
df = pd.read_csv(filepath_or_buffer='/kaggle/input/iris-dataset-extended/iris_extended.csv')
df.head()

,species,elevation,soil_type,sepal_length,sepal_width,petal_length,petal_width,sepal_area,petal_area,sepal_aspect_ratio,...,sepal_to_petal_length_ratio,sepal_to_petal_width_ratio,sepal_petal_length_diff,sepal_petal_width_diff,petal_curvature_mm,petal_texture_trichomes_per_mm2,leaf_area_cm2,sepal_area_sqrt,petal_area_sqrt,area_ratios
0,setosa,161.8,sandy,5.16,3.80,1.64,0.29,19.6080,0.4756,1.357895,...,3.146341,13.103448,3.52,3.51,5.33,18.33,53.21,4.428092,0.689638,41.227923
1,setosa,291.4,clay,5.48,3.87,1.53,0.35,21.2076,0.5355,1.416021,...,3.581699,11.057143,3.95,3.52,5.90,20.45,52.53,4.605171,0.731779,39.603361
2,setosa,144.3,sandy,5.10,3.13,1.47,0.41,15.9630,0.6027,1.629393,...,3.469388,7.634146,3.63,2.72,5.66,24.62,50.25,3.995372,0.776338,26.485814
3,setosa,114.6,clay,4.64,3.28,1.53,0.16,15.2192,0.2448,1.414634,...,3.032680,20.500000,3.11,3.12,4.51,22.91,50.85,3.901179,0.494773,62.169935
4,setosa,110.9,loamy,4.85,2.92,1.26,0.27,14.1620,0.3402,1.660959,...,3.849206,10.814815,3.59,2.65,4.03,21.56,40.57,3.763243,0.583267,41.628454


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   species                          1200 non-null   object 
 1   elevation                        1200 non-null   float64
 2   soil_type                        1200 non-null   object 
 3   sepal_length                     1200 non-null   float64
 4   sepal_width                      1200 non-null   float64
 5   petal_length                     1200 non-null   float64
 6   petal_width                      1200 non-null   float64
 7   sepal_area                       1200 non-null   float64
 8   petal_area                       1200 non-null   float64
 9   sepal_aspect_ratio               1200 non-null   float64
 10  petal_aspect_ratio               1200 non-null   float64
 11  sepal_to_petal_length_ratio      1200 non-null   float64
 12  sepal_to_petal_width

In [4]:
from plotly.express import histogram
for column in df.columns:
    if column not in ['species', 'soil_type']:
        histogram(data_frame=df, x=column, color='species').show()

Clearly the setosa have very different petals but the other two species are harder to differentiate based on the available data.

We have a lot of dimensions; let's do some dimension reduction.

In [5]:
from sklearn.manifold import TSNE
from plotly.express import scatter

def tsne_plot(input_df: pd.DataFrame, columns_to_drop: list):
    # we want our code to be reintrant
    result_df = input_df.copy()
    for column in ['x', 'y']:
        if column in result_df.columns:
            result_df = result_df.drop(columns=[column])
    tsne = TSNE(n_components=2, verbose=1, random_state=2023)
    result_df[['x', 'y']]  = tsne.fit_transform(X=result_df.drop(columns=columns_to_drop))
    for color in ['species', 'soil_type']:
        scatter(data_frame=result_df, x='x', y='y', color=color, ).show()

tsne_plot(input_df=df, columns_to_drop=['species', 'soil_type', ])

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1200 samples in 0.001s...
[t-SNE] Computed neighbors for 1200 samples in 0.109s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1200
[t-SNE] Computed conditional probabilities for sample 1200 / 1200
[t-SNE] Mean sigma: 7.963450
[t-SNE] KL divergence after 250 iterations with early exaggeration: 53.003372
[t-SNE] KL divergence after 1000 iterations: 0.425416


Clearly TSNE easily separates setosa from the other two species; the soil type not so much.

In [6]:
tsne_plot(input_df=df, columns_to_drop=['species', 'soil_type', 'elevation'])

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1200 samples in 0.001s...
[t-SNE] Computed neighbors for 1200 samples in 0.042s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1200
[t-SNE] Computed conditional probabilities for sample 1200 / 1200
[t-SNE] Mean sigma: 2.640785
[t-SNE] KL divergence after 250 iterations with early exaggeration: 59.172050
[t-SNE] KL divergence after 1000 iterations: 0.737762


We get much better separation among our clusters if we drop the elevation, which as we learned above is distributed more or less uniformly across the dataset.

In [7]:
from plotly.express import imshow
imshow(img=df.drop(columns=['species', 'soil_type',]).corr(), height=1000)